<a href="https://colab.research.google.com/github/aafreen2212/AI-Homework/blob/master/Deep_Learning_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
resp = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip')
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()
# ['bbc.classes', 'bbc.docs', 'bbc.mtx', 'bbc.terms']


['Readme.txt', 'day.csv', 'hour.csv']

In this file, a regression problem is solved using deep learning appraoch.

In [ ]:
new_files = zipfile.extractall()

In [ ]:
import pandas as pd
import numpy as np

day = pd.read_csv('/content/day.csv')
hour = pd.read_csv('/content/hour.csv')

In [ ]:
day.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


In [ ]:
hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
hour.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = hour.copy()

In [ ]:
data.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [ ]:
data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [ ]:
#Define x and y
X = data.drop(['instant', 'dteday', 'cnt'], axis=1)
y = data['cnt']
X.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test = sc.transform(x_test)

In [ ]:
X_train.shape

(13903, 14)

In [ ]:
from keras.models import Sequential
from keras.layers import *


In [ ]:
# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
#relu activation function is used t ReLU for short is a piecewise linear function 
#that will output the input directly if it is positive, otherwise, it will output zero. 
#It has become the default activation function for many 
#types of neural networks because a model that uses it is easier to train and often achieves better performance.
model.add(Dense(30, activation = 'relu', input_dim = 14))
#input_dim is the no. of columns
# Adding the second hidden layer
#higher the hidden unit, higher is the accuracy of the model. 
#However, hidden units should not be too big as it would increase the model complexity
model.add(Dense(units = 125, activation = 'relu'))

# Adding the third hidden layer
model.add(Dense(units = 200, activation = 'relu'))
#Regularization parameter: Dropout
#the outputs of a layer under dropout are randomly subsampled, 
#it has the effect of reducing the capacity or thinning the network during training. 
#As such, a wider network, e.g. more nodes, may be required when using dropout.
model.add(Dropout(0.25))
# Adding the output layer
model.add(Dense(units = 1))

In [ ]:
model.compile(optimizer = 'adam',loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])

In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="weights.{epoch:02d}-{mean_absolute_error:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='mean_absolute_error', verbose=1, save_best_only=True, mode='min')


In [ ]:
history = model.fit(x_train, y_train, epochs = 100, callbacks = [checkpoint])

Epoch 1/100
430/435 [============================>.] - ETA: 0s - loss: 16.9743 - mean_absolute_error: 16.9743
Epoch 00001: mean_absolute_error improved from inf to 16.91179, saving model to weights.01-16.91.hdf5
435/435 [==============================] - 1s 2ms/step - loss: 16.9118 - mean_absolute_error: 16.9118
Epoch 2/100
416/435 [===========================>..] - ETA: 0s - loss: 10.0662 - mean_absolute_error: 10.0662
Epoch 00002: mean_absolute_error improved from 16.91179 to 10.05709, saving model to weights.02-10.06.hdf5
435/435 [==============================] - 1s 2ms/step - loss: 10.0571 - mean_absolute_error: 10.0571
Epoch 3/100
420/435 [===========================>..] - ETA: 0s - loss: 10.1746 - mean_absolute_error: 10.1746
Epoch 00003: mean_absolute_error did not improve from 10.05709
435/435 [==============================] - 1s 2ms/step - loss: 10.1654 - mean_absolute_error: 10.1654
Epoch 4/100
421/435 [============================>.] - ETA: 0s - loss: 10.0304 - mean_absolu

In [ ]:
from tensorflow.keras.models import load_model
model_new = load_model('/content/weights.88-9.07.hdf5')


In [ ]:
model_new.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                450       
_________________________________________________________________
dense_5 (Dense)              (None, 125)               3875      
_________________________________________________________________
dense_6 (Dense)              (None, 200)               25200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 29,726
Trainable params: 29,726
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_pred = model_new.predict(x_test)

In [ ]:
from sklearn.metrics import mean_absolute_error

error = mean_absolute_error(y_test, y_pred)
print(error)
#The error is 99.18

2.6435587233311835


In [ ]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test, y_pred)
print('Model accuracy is = {:0.6f}'.format(accuracy))

Model accuracy is = 0.999576


In [ ]:
from keras.models import load_model
#The shape of weights for the selected model
model_4 = load_model('/content/weights.best.hdf5')
for layer in model.layers:
  if len(layer.weights) > 0:
    print(layer.name, layer.weights[0].shape)

dense_81 (14, 30)
dense_82 (30, 32)
dense_83 (32, 32)
dense_84 (32, 1)



At each epoch, the model completes training on all the observations of the train set. Therefore, at 4th epoch, model has gone through the train set for 3 times. In order to perform backpropagation, forward propagation should be completed. The values from the last layer of the nueral networks got from the forward propagation are considered. dJ/dw1 (for each weights of input layer i.e. 14 cols) and dJ/db where J is the cost function. These gradients could be used in the adam optimizers to improve the weights and biases where the cost function is the minimum. The following cost function shows the weight matrices for layer3, layer2 and layer1 of the model. The weights are updated such that the cost function is the lowest.



In [ ]:
#The weights in layer 3 would be zero since it consists of the outputs
model_4_3 = load_model('/content/weights.04-10.05.hdf5')
weight_layer_4_3=model_4_3.layers[3].get_weights()
print(weight_layer_4_3)

[]


In [ ]:
#Weight Matrix of 2nd layer of epoch 4
weight_layer_4_2=model_4_1.layers[2].get_weights()
print(weight_layer_4_2)

[array([[-0.00458941,  0.04921785, -0.08894711, ..., -0.04426719,
         0.05454879,  0.08504193],
       [-0.06793217,  0.17300902, -0.07112951, ..., -0.1079075 ,
        -0.06335767,  0.03065375],
       [ 0.0791788 , -0.04300268,  0.10687821, ..., -0.06552172,
        -0.14840503,  0.16467042],
       ...,
       [-0.0272062 , -0.04702072,  0.10114998, ..., -0.05975083,
         0.05803816, -0.0698769 ],
       [-0.00776919, -0.0016284 , -0.15676688, ..., -0.05249582,
         0.09171531,  0.09002262],
       [-0.02660254, -0.11120652, -0.01088137, ..., -0.05030262,
         0.03885168,  0.04014947]], dtype=float32), array([-0.0281083 ,  0.02904174,  0.00230422,  0.02941417,  0.02290462,
       -0.02293523, -0.03086608,  0.01144583,  0.03060494,  0.01677356,
       -0.02704727,  0.        ,  0.03764931,  0.00875828,  0.0437961 ,
       -0.01937194,  0.04052368, -0.0117595 , -0.01157436, -0.0470533 ,
        0.03466826,  0.02216711,  0.03011546, -0.02730229, -0.01722995,
       -0.

Now consider two layers: Layer 1 and Layer2 for the best model (88 epoch)

In Layer 1, the input weights from 0th layer enters and is subjected to the ReLu activation function that causes non-linearity in the corresponding output of layer 1. The weight matrix for layer 1 is shown below. Since layer 1 consists of 125 hidden units, the 125 activation functions are applied to different inputs in order to obtain the output Z1,...,Z125 where Z is the output result from activation function (ReLu in this case).

In [ ]:
layer_model = load_model('/content/weights.88-9.07.hdf5')

In [ ]:
#The dimensions of this weight matrix would W[n0, n1] where n is the no. of hidden units. 
#Therefore, W is of shape [30, 125]
weight_layer1=layer_model.layers[1].get_weights()
weight_layer1

[array([[-0.00355187, -0.21751715, -0.10569739, ..., -0.00188616,
         -0.0109664 ,  0.18243833],
        [ 0.01861395, -0.05245482, -0.00219852, ...,  0.01996835,
         -0.04726018, -0.27175313],
        [ 0.07646588,  0.13112396, -0.07909629, ...,  0.14895868,
          0.16435474,  0.04438773],
        ...,
        [-0.02012443, -0.163281  ,  0.13718003, ..., -0.14855516,
          0.07398589, -0.0188133 ],
        [-0.09043436,  0.00047712, -0.12893668, ..., -0.16364358,
         -0.19972505, -0.209863  ],
        [-0.08225781, -0.15726183, -0.0965673 , ..., -0.19278766,
         -0.09948368,  0.03678462]], dtype=float32),
 array([ 0.2347689 , -0.08649663, -0.11545159, -0.18231101, -0.21604803,
        -0.21787594, -0.10799897, -0.28542998, -0.24323739,  0.01999495,
        -0.18727298, -0.11569253,  0.07830626, -0.17924109, -0.23356818,
        -0.15116297, -0.13398866, -0.14642449, -0.22420427, -0.27072266,
        -0.16865166, -0.05801897, -0.18878292, -0.30473495, -0.103

In Layer 2, the input weights from 1st layer enters and is subjected to the ReLu activation function that causes non-linearity in the corresponding output of layer 2. The weight matrix for layer 2 is shown below. Since layer 1 consists of 30 hidden units, the 30 activation functions are applied to different inputs in order to obtain the output Z200,...,Z200.

In [ ]:
weight_layer_2=layer_model.layers[2].get_weights()
print(weight_layer_2)

[array([[-0.06904548, -0.04293536,  0.06275894, ..., -0.04426719,
         0.05454879, -0.00467303],
       [-0.02284556,  0.16231243, -0.03582188, ..., -0.1079075 ,
        -0.06335767, -0.00346581],
       [ 0.07445255, -0.04596286,  0.1022029 , ..., -0.06552172,
        -0.14840503,  0.14089508],
       ...,
       [-0.07680734, -0.01629007,  0.15614364, ..., -0.08882368,
         0.05803816, -0.0644895 ],
       [-0.09181899, -0.0539087 , -0.1941104 , ..., -0.05249582,
         0.06950644,  0.05294609],
       [ 0.29835275, -0.09052721,  0.30216485, ..., -0.05030262,
         0.00829348, -0.16802594]], dtype=float32), array([ 0.20210642, -0.00418124,  0.13864769,  0.01497327,  0.02495383,
       -0.07555717, -0.01009439, -0.0310457 ,  0.02063909,  0.01153323,
       -0.09370562, -0.06125431, -0.0010783 , -0.06664619, -0.00818328,
       -0.11395684, -0.05478109, -0.04326407, -0.02724057, -0.16712552,
       -0.00630098, -0.07779337,  0.0303162 , -0.10858808, -0.08742786,
        0.

The third layer is the output layer that consists of predictions and therefore lacks weight matrices.

In [ ]:
weight_layer_3=layer_model.layers[3].get_weights()
print(weight_layer_3)

[]
